In [1]:
import pandas as pd
import numpy as np
import re
pd.options.mode.chained_assignment = None 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [2]:
dataset = pd.io.parsers.read_csv('a1_d3.txt', sep='\t',names=['data','value'])
dataset['data'] = dataset['data'].str.lower()
stop = ["ive","iam","im","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
alphabets = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
dataset['data'] = dataset['data'].str.replace(r'[-,\.&!\+\'0-9;"\(\)/#@$*%^&?~`:'']','',regex=False)
dataset['data'] = list(map(str.split, dataset['data']))
dataset['data'] = dataset['data'].apply(lambda x: [item for item in x if item not in stop])
dataset['data'] = dataset['data'].apply(lambda x: [item for item in x if item not in alphabets])
dataset['data'] = dataset['data'].apply(lambda x: [stemmer.stem(y) for y in x]) 
# dataset = dataset.sample(frac=1)
dataset = np.array_split(dataset,5)

In [3]:
val=0
array = np.zeros(5)
for temp in range(5):
    frames = [dataset[0],dataset[1],dataset[2],dataset[3],dataset[4]]
    frames.pop(temp)
    train_set = pd.concat(frames)
    test_set = dataset[temp]
    x_train = train_set['data']
    y_train = train_set['value']
    x_test = test_set['data']
    y_test = test_set['value']
    zeroes = (y_train==0).sum(axis=0)
    ones = (y_train==1).sum(axis=0)
    x_train1 = x_train[y_train==1]
    x_train0 = x_train[y_train==0]
    dict1 = {}
    for data in x_train1:
        for item in data:
            dict1[item] = 0
    for data in x_train1:
        for item in data:
            dict1[item] +=1
    dict1 = pd.DataFrame(dict1.items(),columns=['word','freq'])
    unique_words1 = len(dict1.index)
    alpha = 1
    total_words1 = np.sum(dict1['freq'])
    temp1 = total_words1 + unique_words1
#     dict1['prob'] = (dict1['freq'] + alpha)/(temp1)
    dict1['prob'] = dict1['freq']/total_words1
    dict = {}
    for data in x_train0:
        for item in data:
            dict[item] = 0
    for data in x_train0:
        for item in data:
                dict[item] +=1
    dict0 = pd.DataFrame(dict.items(),columns=['word','freq'])
    unique_words0 = len(dict0.index)
    total_words0 = np.sum(dict0['freq'])
    temp0 = total_words0 + unique_words0
#     dict0['prob'] = (dict0['freq'] + alpha)/(temp0)
    dict0['prob'] = dict0['freq']/total_words0
    x_test = pd.DataFrame(x_test,columns=['data','prob1','prob0'])
    x_test['prob1'] = 0.0
    x_test['prob0'] = 0.0
    x_test = x_test.reset_index(inplace=False,drop=True)
    i=0
    for data in x_test['data']:
        for item in data:
            if item in dict1.values:
                x_test['prob1'][i] += np.log(dict1.loc[dict1['word']==item,'prob'])
            else:
                x_test['prob1'][i] += np.log(1/(temp1))
        i += 1 
    i=0
    for data in x_test['data']:
        for item in data:
            if item in dict0.values:
                x_test['prob0'][i] += np.log(dict0.loc[dict0['word']==item,'prob'])
            else:
                x_test['prob0'][i] += np.log(1/(temp0))
        i += 1
    zeroes = (y_train==0).sum(axis=0)/y_train.size
    ones = (y_train==1).sum(axis=0)/y_train.size
    x_test['prob1'] = x_test['prob1'] 
    x_test['prob0'] = x_test['prob0'] 
    ans = x_test['prob1']>x_test['prob0']
    ans = ans.astype(int)
    ans = np.array(ans)
    y_test = np.array(y_test)
    arr = (ans==y_test).sum()
    print(arr/200)
    array[temp] = arr/y_test.size
print(np.mean(array)*100,"+/-",np.std(array)*100)

0.755
0.785
0.695
0.675
0.75
73.2 +/- 4.069397989875161
